In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00


In [2]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 6.7 MB/s eta 0:00:00


In [3]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.9 MB/s eta 0:00:00


In [4]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import load_dataset

import re
import numpy as np
import torch
from torch import optim
import emoji
from emoji import demojize
import nltk
from nltk.corpus import stopwords
import contractions

nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
df = pd.read_csv('/content/gdrive/MyDrive/Disaster Tweets/data/train.csv')

In [7]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
df.shape

(7613, 5)

In [9]:
df['target'].value_counts(normalize='true')

0    0.57034
1    0.42966
Name: target, dtype: float64

In [10]:
def preprocess_text(text):
    """
    Preprocesses a tweet text for further analysis.

    Args:
        text: The text of the tweet to be preprocessed.

    Returns:
        str: The preprocessed tweet text.
    """

    # Expand contractions
    text = contractions.fix(text)

    # Remove URLs
    url_pattern = r"(https?://)?(www\.)?(\S+\.\S+)(/\S*)?"
    text = re.sub(url_pattern, "", text)

    # Replace emojis with their meaning
    text = emoji.demojize(text)

    # Remove punctuations
    punctuation_pattern = r"[^\w\s]"
    text = re.sub(punctuation_pattern, "", text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_word = [word for word in words if word.lower() not in stop_words]
    text = " ".join(filtered_word)

    # Lowercase the text
    text = text.lower()

    return text


In [11]:
data = df[['text', 'target']]

In [12]:
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
data['text'] = data['text'].apply(preprocess_text)

In [14]:
data.head()

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked shelter place notified officer...,1
3,13000 people receive wildfires evacuation orde...,1
4,got sent photo ruby alaska smoke wildfires pou...,1


In [15]:
data['target'] = data['target'].apply(lambda x: 1 if x==0 else 0)

In [16]:
data.head()

,text,target
0,deeds reason earthquake may allah forgive us,0
1,forest fire near la ronge sask canada,0
2,residents asked shelter place notified officer...,0
3,13000 people receive wildfires evacuation orde...,0
4,got sent photo ruby alaska smoke wildfires pou...,0


In [26]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
EPOCHS = 10
learning_rate = 1e-05

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [20]:
model.config.num_labels = 2

In [21]:
def tokenize_text(text):
  encoded_text = tokenizer(text, return_tensors = 'pt', padding=True)
  return encoded_text.to(device)

In [22]:
training_data = [tokenize_text(text) for text in data['text']]
training_labels = data['target']

In [23]:
def calculate_acc(pred, targets, data):
  n_correct = (pred == targets).sum().item()
  accuracy = n_correct / len(data)

  return np.round((accuracy * 100), 5)

In [24]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [27]:
for epoch in range(EPOCHS):
  outputs = []
  model.train()

  for text, label in zip(training_data, training_labels):

    output = model(**text)
    loss = criterion(output.logits, torch.tensor([label]).to(device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    outputs.append(torch.argmax(output.logits).item())

  accuracy = calculate_acc(outputs, training_labels, data)
  print(f'epoch: {epoch} has an Accuracy of {accuracy}')

epoch: 0 has an Accuracy of 80.58584
epoch: 1 has an Accuracy of 86.49678
epoch: 2 has an Accuracy of 90.85774
epoch: 3 has an Accuracy of 93.18271
epoch: 4 has an Accuracy of 94.62761
epoch: 5 has an Accuracy of 95.63904
epoch: 6 has an Accuracy of 96.00683
epoch: 7 has an Accuracy of 96.28267
epoch: 8 has an Accuracy of 96.75555
epoch: 9 has an Accuracy of 97.00512


In [31]:
for epoch in range(5):
  outputs = []
  model.train()

  for text, label in zip(training_data, training_labels):

    output = model(**text)
    loss = criterion(output.logits, torch.tensor([label]).to(device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    outputs.append(torch.argmax(output.logits).item())

  accuracy = calculate_acc(outputs, training_labels, data)
  print(f'epoch: {epoch} has an Accuracy of {accuracy}')

epoch: 0 has an Accuracy of 97.09707
epoch: 1 has an Accuracy of 97.12334
epoch: 2 has an Accuracy of 97.38605
epoch: 3 has an Accuracy of 97.38605
epoch: 4 has an Accuracy of 97.2941


In [32]:
def predict_sentiment(text):
    encoded_text = tokenize_text(text)
    #input_ids = torch.tensor([encoded_text]).to(device)
    with torch.no_grad():
        outputs = model(**encoded_text)
        pred = torch.argmax(outputs.logits).item()
    # Convert prediction to sentiment label (e.g., "positive", "negative")
    return pred


In [33]:
predicted_sentiments = []
for text in data["text"]:
    predicted_sentiment = predict_sentiment(text)  # Get the sentiment value
    predicted_sentiments.append(predicted_sentiment)

data["predict"] = predicted_sentiments

In [34]:
from sklearn.metrics import confusion_matrix

# Assuming `data['target']` contains true labels and `predicted_sentiments` contains predictions
confusion_matrix = confusion_matrix(data['target'], data['predict'])
print(confusion_matrix)

[[3229   42]
 [ 162 4180]]
